In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "1" 
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [4]:
# import torch
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#torch.backends.cuda.matmul.allow_tf32 = True


# print('Device:', device)
# print('Current cuda device:', torch.cuda.current_device())
# print('Count of using GPUs:', torch.cuda.device_count())

In [7]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
device

device(type='cuda')

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
import os
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler, SGD, Adam
from torch.utils.data import Subset, Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import time
from tqdm.auto import tqdm
from scipy.io import wavfile
#from vggm import VGGM
import argparse
from scipy.signal import lfilter, stft
import librosa
from numba import jit
from torch.utils.tensorboard import SummaryWriter
import soundfile as sf

In [11]:
def loadWAV(filename, max_frames, evalmode=True, num_eval=1):

    # Maximum audio length
    max_audio = max_frames * 160 + 240

    # Read wav file and convert to torch tensor
    audio, sample_rate = librosa.load(filename,sr=16000)
    #audio = audio.cpu().detach().numpy().squeeze()
    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]

    if evalmode:
        startframe = numpy.linspace(0,audiosize-max_audio,num=num_eval)
    else:
        startframe = numpy.array([numpy.int64(random.random()*(audiosize-max_audio))])
    
    feats = []
    if evalmode and max_frames == 0:
        feats.append(audio)
    else:
        for asf in startframe:
            feats.append(audio[int(asf):int(asf)+max_audio])

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return torch.from_numpy(feat);

In [12]:
import soundfile
import numpy
import random
from sklearn.preprocessing import MinMaxScaler


class PreEmphasis(torch.nn.Module):

    def __init__(self, coef: float = 0.97):
        super().__init__()
        self.coef = coef
        self.register_buffer(
            'flipped_filter', torch.FloatTensor([-self.coef, 1.]).unsqueeze(0).unsqueeze(0)
        )

    def forward(self, input: torch.tensor) -> torch.tensor:
        input = input.unsqueeze(1)
        input = F.pad(input, (1, 0), 'reflect')
        return F.conv1d(input, self.flipped_filter).squeeze(1)
    
import soundfile as sf
import audioread
import torchaudio
from pydub import AudioSegment

class AudioDataset(Dataset):
    def __init__(self, csv_file,label, data_dir, croplen=32240, is_train=True):
        if isinstance(csv_file, str):
            csv_file=pd.read_csv(csv_file)
        assert isinstance(csv_file, pd.DataFrame), "Invalid csv path or dataframe"
        self.x=csv_file['Path'].values
        self.y=label
        self.data_dir=data_dir
        self.is_train=is_train
        self.croplen=croplen
        self.torchfbank = torch.nn.Sequential(
            #PreEmphasis(),            
            torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_fft=512, win_length=400, hop_length=160, \
                                                 f_min = 20, f_max = 7600, window_fn=torch.hamming_window, n_mels=80),
            )

    def __len__(self):
        return len(self.x)
    

    def __getitem__(self, idx):
        label=self.y[idx] 
        
        #aro = audioread.ffdec.FFmpegAudioFile(os.path.join(self.data_dir,self.x[idx]))
        #audio,sr=librosa.load(os.path.join(self.data_dir,self.x[idx]))
        '''audio= AudioSegment.from_file(os.path.join(self.data_dir,self.x[idx]))
        audio = audio.get_array_of_samples()

        audio = np.array(audio).astype(np.float32)
        audio, index = torchaudio.load(os.path.join(self.data_dir,self.x[idx]))
        audio = audio.squeeze()'''
        audio = loadWAV(os.path.join(self.data_dir,self.x[idx]),200,evalmode=False)
        #audio,sr = librosa.load(os.path.join(self.data_dir,self.x[idx]))
        #audio = audio.squeeze()
        '''
        if(self.is_train):
            start=np.random.randint(0,audio.shape[0]-self.croplen+1)
            audio=audio[start:start+self.croplen]'''
        
        #audio = np.abs(audio)
        
        #audio = audio.reshape(1,audio.shape[0])
        #audio = librosa.feature.melspectrogram(y=audio,sr =16000,n_fft = 400 , hop_length = 160, power = 2.0, n_mels = 40 , fmax=7600,fmin=20)
        #audio = torch.from_numpy(audio)
        #audio = audio.log()   
        #scaler_data = MinMaxScaler()
        #scaler_data.fit(audio.reshape(-1,1))
        #audio = scaler_data.transform(audio.reshape(-1,1))
            
        return audio.squeeze(), label

In [13]:
# train_dataset = AudioDataset(df_F, classes, DATA_DIR, 32240, True)
# train_loader = DataLoader(train_dataset, batch_size=512,  num_workers=40,shuffle = True,drop_last = False,pin_memory=True)

In [14]:
import math, torch, torchaudio
import torch.nn as nn
import torch.nn.functional as F


class SEModule(nn.Module):
    def __init__(self, channels, bottleneck=128):
        super(SEModule, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),
            nn.Conv1d(channels, bottleneck, kernel_size=1, padding=0),
            nn.ReLU(),
            # nn.BatchNorm1d(bottleneck), # I remove this layer
            nn.Conv1d(bottleneck, channels, kernel_size=1, padding=0),
            nn.Sigmoid(),
            )

    def forward(self, input):
        x = self.se(input)
        return input * x

class Bottle2neck(nn.Module):

    def __init__(self, inplanes, planes, kernel_size=None, dilation=None, scale = 8):
        super(Bottle2neck, self).__init__()
        width       = int(math.floor(planes / scale))
        self.conv1  = nn.Conv1d(inplanes, width*scale, kernel_size=1)
        self.bn1    = nn.BatchNorm1d(width*scale)
        self.nums   = scale -1
        convs       = []
        bns         = []
        num_pad = math.floor(kernel_size/2)*dilation
        for i in range(self.nums):
            convs.append(nn.Conv1d(width, width, kernel_size=kernel_size, dilation=dilation, padding=num_pad))
            bns.append(nn.BatchNorm1d(width))
        self.convs  = nn.ModuleList(convs)
        self.bns    = nn.ModuleList(bns)
        self.conv3  = nn.Conv1d(width*scale, planes, kernel_size=1)
        self.bn3    = nn.BatchNorm1d(planes)
        self.relu   = nn.ReLU()
        self.width  = width
        self.se     = SEModule(planes)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.relu(out)
        out = self.bn1(out)

        spx = torch.split(out, self.width, 1)
        for i in range(self.nums):
            if i==0:
                sp = spx[i]
            else:
                sp = sp + spx[i]
            sp = self.convs[i](sp)
            sp = self.relu(sp)
            sp = self.bns[i](sp)
            if i==0:
                out = sp
            else:
                out = torch.cat((out, sp), 1)
        out = torch.cat((out, spx[self.nums]),1)

        out = self.conv3(out)
        out = self.relu(out)
        out = self.bn3(out)
        
        out = self.se(out)
        out += residual
        return out 

class PreEmphasis(torch.nn.Module):

    def __init__(self, coef: float = 0.97):
        super().__init__()
        self.coef = coef
        self.register_buffer(
            'flipped_filter', torch.FloatTensor([-self.coef, 1.]).unsqueeze(0).unsqueeze(0)
        )

    def forward(self, input: torch.tensor) -> torch.tensor:
        input = input.unsqueeze(1)
        input = F.pad(input, (1, 0), 'reflect')
        return F.conv1d(input, self.flipped_filter).squeeze(1)

class FbankAug(nn.Module):

    def __init__(self, freq_mask_width = (0, 8), time_mask_width = (0, 10)):
        self.time_mask_width = time_mask_width
        self.freq_mask_width = freq_mask_width
        super().__init__()

    def mask_along_axis(self, x, dim):
        original_size = x.shape
        batch, fea, time = x.shape
        if dim == 1:
            D = fea
            width_range = self.freq_mask_width
        else:
            D = time
            width_range = self.time_mask_width

        mask_len = torch.randint(width_range[0], width_range[1], (batch, 1), device=x.device).unsqueeze(2)
        mask_pos = torch.randint(0, max(1, D - mask_len.max()), (batch, 1), device=x.device).unsqueeze(2)
        arange = torch.arange(D, device=x.device).view(1, 1, -1)
        mask = (mask_pos <= arange) * (arange < (mask_pos + mask_len))
        mask = mask.any(dim=1)

        if dim == 1:
            mask = mask.unsqueeze(2)
        else:
            mask = mask.unsqueeze(1)
            
        x = x.masked_fill_(mask, 0.0)
        return x.view(*original_size)

    def forward(self, x):    
        x = self.mask_along_axis(x, dim=2)
        x = self.mask_along_axis(x, dim=1)
        return x

class ECAPA_TDNN(nn.Module):

    def __init__(self, C):

        super(ECAPA_TDNN, self).__init__()

        self.torchfbank = torch.nn.Sequential(
            PreEmphasis(),            
            torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_fft=512, win_length=400, hop_length=160, \
                                                 f_min = 20, f_max = 7600, window_fn=torch.hamming_window, n_mels=80),
            )

        self.specaug = FbankAug() # Spec augmentation

        self.conv1  = nn.Conv1d(80, C, kernel_size=5, stride=1, padding=2)
        self.relu   = nn.ReLU()
        self.bn1    = nn.BatchNorm1d(C)
        self.layer1 = Bottle2neck(C, C, kernel_size=3, dilation=2, scale=8)
        self.layer2 = Bottle2neck(C, C, kernel_size=3, dilation=3, scale=8)
        self.layer3 = Bottle2neck(C, C, kernel_size=3, dilation=4, scale=8)
        # I fixed the shape of the output from MFA layer, that is close to the setting from ECAPA paper.
        self.layer4 = nn.Conv1d(3*C, 1536, kernel_size=1)
        self.attention = nn.Sequential(
            nn.Conv1d(4608, 256, kernel_size=1),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Tanh(), # I add this layer
            nn.Conv1d(256, 1536, kernel_size=1),
            nn.Softmax(dim=2),
            )
        self.bn5 = nn.BatchNorm1d(3072)
        self.fc6 = nn.Linear(3072, 192)
        self.bn6 = nn.BatchNorm1d(192)
        self.fc7 = nn.Linear(192,5994)


    def forward(self, x, aug):
        with torch.no_grad():
            x = self.torchfbank(x)+1e-6
            x = x.log()   
            x = x - torch.mean(x, dim=-1, keepdim=True)
            if aug == True:
                x = self.specaug(x)

        x = self.conv1(x)
        x = self.relu(x)
        x = self.bn1(x)

        x1 = self.layer1(x)
        x2 = self.layer2(x+x1)
        x3 = self.layer3(x+x1+x2)

        x = self.layer4(torch.cat((x1,x2,x3),dim=1))
        x = self.relu(x)

        t = x.size()[-1]

        global_x = torch.cat((x,torch.mean(x,dim=2,keepdim=True).repeat(1,1,t), torch.sqrt(torch.var(x,dim=2,keepdim=True).clamp(min=1e-4)).repeat(1,1,t)), dim=1)
        
        w = self.attention(global_x)
    
        mu = torch.sum(x * w, dim=2)
        sg = torch.sqrt( ( torch.sum((x**2) * w, dim=2) - mu**2 ).clamp(min=1e-4) )

        x = torch.cat((mu,sg),1)
        x = self.bn5(x)
        emb = self.fc6(x)
        out = self.bn6(emb)
        #out = nn.ReLU()(out)
        #out = self.fc7(out)
        #emb = torch.nn.functional.dropout(emb,p=0.5,training=True)
        return emb,out

In [15]:
model = ECAPA_TDNN(1024).to(device)
model.load_state_dict(torch.load('/data/hypark/VC/Auto_Nansy/VC/ecapa_tdnn/tdnn.pt'))

<All keys matched successfully>

# Train

In [ ]:
def ppdf(df_F):
    df_F['Label']=df_F['Path'].str.split("/", n=1, expand=True)[0].str.replace("id","")
    df_F['Label']=df_F['Label'].astype(dtype=float)
    # print(df_F.head(20))
    #df_F['Path']=df_F['Path'].str.replace(".m4a","")
    return df_F

df_F=pd.read_csv('/workspace/data/chgo/'+"wav_train_list.txt", sep=" ", names=["Set","Path"] )

df_F[:]

index = 0
classes = []
for i in range(0,len(df_F['Set'])-1,1):
    if df_F['Set'][i] == df_F['Set'][i+1] :
        classes.append(index)
    else :
        index = index +1
        classes.append(index)
classes.append(5993)

In [41]:
test_Datasets[1][0].unsqueeze(dim=0).shape

torch.Size([1, 51133])

In [42]:
torch.manual_seed(777)

In [43]:
import torch.optim as optimizer
criterion = nn.CrossEntropyLoss().to(device)   
optimizer = optimizer.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)
scaler = torch.cuda.amp.GradScaler()

In [24]:
from tqdm import tqdm


train_ACC = []
train_loss = []
model.load_state_dict(torch.load('/workspace/data/chgo/tdnn.pt'))

for epoch in range(30):
    total_loss=0
    correct = 0
    val_correct = 0
    train_acc = 0
    vaild_acc = 0
    best = 0
    

    pred = 0
    for data , label in tqdm(train_loader,total = len(train_loader)):
        model.train()
        optimizer.zero_grad()
        data = data.to(device).float()
        label = label.to(device)
        _,output = model(data,True)
        loss = criterion(output, label)
            
        prediction = output.argmax(dim=1,keepdim=True)
        correct += prediction.eq(label.view_as(prediction)).sum().item()
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
    train_acc = 100 * correct / len(train_loader.dataset)
    train_ACC.append(train_acc)
    train_loss.append(total_loss/len(train_loader.dataset))
    scheduler.step()
    
        #torch.save(model.state_dict(),f'/data/chgo/voxceleb2_weight/vgg_x-vector{epoch}.pt') ### resnet
    torch.save(model.state_dict(),f'/workspace/data/chgo/tdnn.pt') ### vggvox
    print("epoch :",epoch , "train_acc : " ,train_acc , "loss : ", total_loss/len(train_loader.dataset))
    '''
    
    with torch.no_grad():
        model.eval()
        
        for signal , target in tqdm(vaild_loader,total = len(vaild_loader)):
            signal = signal.to(device).float()
            target = target.to(device)
            out = model(signal)
            predict = out.argmax(dim=1,keepdim=True)
            val_correct += predict.eq(target.view_as(predict)).sum().item()
          
        vaild_acc = 100 * val_correct / len(vaild_loader.dataset)
        
        if vaild_acc > best :
            best = vaild_acc
            torch.save(model.state_dict(),'/workspace/data/chgo/kict_classifier test file/lstm_weight.pt')
            
      
        print("vaild_acc :",vaild_acc)
        
        '''

100%|███████████████████████████████████████| 2133/2133 [24:20<00:00,  1.46it/s]


epoch : 0 train_acc :  94.0644262089415 loss :  0.0006966863571851087


  0%|                                        | 1/2133 [00:06<4:00:11,  6.76s/it]
Exception in thread Thread-7 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/pin_memory.py", line 49, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/pin_memory.py", line 26, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.10/dist-packages/torch/multiprocessing/reductions.py", line 305, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.10/multiprocessing/resource_sharer.py", line 57, in detach
    

KeyboardInterrupt: 

# Speaker Embedding

In [16]:
class testDataset(Dataset):
    def __init__(self, x):

        self.x=x

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        # target = self.x[idx].split("/")[6]
        audio, sr=librosa.load(self.x[idx],sr=16000)
        
        return torch.from_numpy(audio).squeeze()

In [17]:
from glob import glob
data_path = '/data/hypark/VC/data/VCTK/wav48/'
speaker_name = sorted(os.listdir(data_path))

speakers = []
for name in speaker_name:
    test_dir = 1
    test_dir = glob(os.path.join(data_path, name, '*.wav'))
    test_Datasets = testDataset(test_dir)
    
    test_loader = DataLoader(test_Datasets, batch_size=1, shuffle=False, drop_last=True)
    
    Data1 = []
    
    with torch.no_grad():
        model.eval()
        
        for i  in tqdm(test_loader, total=len(test_loader)):
            feature1, _ = model(i.float().to(device), False)
            feature1 = F.normalize(feature1, p=2, dim=1)
            Data1.extend(feature1.cpu().detach().numpy().reshape(1,192))
            
        Data1 = np.array(Data1)
        emb = np.mean(Data1, axis=0)

    utterances = []
    utterances.append(name)
    utterances.append(emb)
    utterances.extend(test_dir)
    speakers.append(utterances)

  0%|          | 0/231 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [260]:
dir_path = '/data/hypark/VC/Auto_Nansy/wav2vec/preprocess_data/VCTK/'
import pickle
with open(os.path.join(dir_path, 'perturb_f.pkl'), 'wb') as handle:
    pickle.dump(speakers, handle)